## Build an MTH5 and Operate the Aurora Pipeline

This notebook pulls MT miniSEED data from the IRIS Dataselect web service and produces MTH5 out of it. It outlines the process of making an MTH5 file, generating a processing config, and running the Aurora processor.

It assumes that aurora, mth5, and mt_metadata have all been installed.

In this "new" version, the workflow has changed somewhat.  

1. The process_mth5 call works with a dataset dataframe, rather than a single run_id
2. The config object is now based on the mt_metadata.base Base class
3. Remote reference processing is supported (at least in theory)

### Flow of this notebook

Section 1: Here we do imports and construct a table of the data that we will access to build the mth5.  Note that there is no explanation here as to the table source -- a future update can show how to create such a table from IRIS data_availability tools

Seciton 2: the metadata and the data are accessed, and the mth5 is created and stored.

Section 3: Aurora is used to process the data

In [1]:
# Required imports for the program. 
from pathlib import Path
import pandas as pd
from mth5.clients.make_mth5 import MakeMTH5
from mth5 import mth5, timeseries
from mth5.utils.helpers import initialize_mth5
from mt_metadata.utils.mttime import get_now_utc, MTime
from aurora.config import BANDS_DEFAULT_FILE
from aurora.config.config_creator import ConfigCreator
from aurora.pipelines.process_mth5 import process_mth5
from aurora.transfer_function.kernel_dataset import KernelDataset
from aurora.pipelines.run_summary import RunSummary

2022-07-29 14:44:52,204 [line 135] mth5.setup_logger - INFO: Logging file can be found /home/kkappler/software/irismt/mth5/logs/mth5_debug.log


Build an MTH5 file from information extracted by IRIS

In [2]:
# Set path so MTH5 file builds to current working directory. 
default_path = Path().cwd()
default_path

PosixPath('/home/kkappler/software/irismt/aurora/docs/examples')

In [3]:
# Initialize the Make MTH5 code. 
mth5_version = '0.1.0'
mth5_version = '0.2.0'
m = MakeMTH5(mth5_version=mth5_version)
m.client = "IRIS"

### Specify the data to access from IRIS

Note that here we explicitly prescribe the data, but this dataframe could be built from IRIS data availability tools in a programatic way

In [4]:
# Generate data frame of FDSN Network, Station, Location, Channel, Startime, Endtime codes of interest

CAS04LQE = ['8P', 'CAS04', '', 'LQE', '2020-06-02T19:00:00', '2020-07-13T19:00:00']
CAS04LQN = ['8P', 'CAS04', '', 'LQN', '2020-06-02T19:00:00', '2020-07-13T19:00:00']
CAS04BFE = ['8P', 'CAS04', '', 'LFE', '2020-06-02T19:00:00', '2020-07-13T19:00:00']
CAS04BFN = ['8P', 'CAS04', '', 'LFN', '2020-06-02T19:00:00', '2020-07-13T19:00:00']
CAS04BFZ = ['8P', 'CAS04', '', 'LFZ', '2020-06-02T19:00:00', '2020-07-13T19:00:00']

request_list = [CAS04LQE, CAS04LQN, CAS04BFE, CAS04BFN, CAS04BFZ]

# Turn list into dataframe
request_df =  pd.DataFrame(request_list, columns=m.column_names)

In [5]:
# Inspect the dataframe
print(request_df)

  network station location channel                start                  end
0      8P   CAS04              LQE  2020-06-02T19:00:00  2020-07-13T19:00:00
1      8P   CAS04              LQN  2020-06-02T19:00:00  2020-07-13T19:00:00
2      8P   CAS04              LFE  2020-06-02T19:00:00  2020-07-13T19:00:00
3      8P   CAS04              LFN  2020-06-02T19:00:00  2020-07-13T19:00:00
4      8P   CAS04              LFZ  2020-06-02T19:00:00  2020-07-13T19:00:00


In [6]:
# Request the inventory information from IRIS
inventory = m.get_inventory_from_df(request_df, data=False)

In [7]:
# Inspect the inventory
inventory

(Inventory created at 2022-07-29T21:44:59.434092Z
	Created by: ObsPy 1.2.2
		    https://www.obspy.org
	Sending institution: MTH5
	Contains:
		Networks (1):
			8P
		Stations (1):
			8P.CAS04 (Corral Hollow, CA, USA)
		Channels (8):
			8P.CAS04..LFZ, 8P.CAS04..LFN, 8P.CAS04..LFE, 8P.CAS04..LQN (2x), 
			8P.CAS04..LQE (3x),
 0 Trace(s) in Stream:
)

Builds an MTH5 file from the user defined database. 
Note: Intact keeps the MTH5 open after it is done building


With the mth5 object set, we are ready to actually request the data from the fdsn client (IRIS) and save it to an MTH5 file.  This process builds an MTH5 file and can take some time depending on how much data is requested.  

Note: interact keeps the MTH5 open after it is done building


In [8]:
interact = True
mth5_object = m.make_mth5_from_fdsnclient(request_df, interact=interact)

2022-07-29 14:45:03,876 [line 591] mth5.mth5.MTH5.open_mth5 - WARNING: 8P_CAS04.h5 will be overwritten in 'w' mode
2022-07-29 14:45:04,372 [line 656] mth5.mth5.MTH5._initialize_file - INFO: Initialized MTH5 0.2.0 file /home/kkappler/software/irismt/aurora/docs/examples/8P_CAS04.h5 in mode w
2022-07-29 14:45:11,551 [line 121] mt_metadata.base.metadata.station.add_run - WARNING: Run a is being overwritten with current information


2022-07-29T14:45:11 [line 136] obspy_stages.create_filter_from_stage - INFO: Converting PoleZerosResponseStage electric_si_units to a CoefficientFilter
2022-07-29T14:45:11 [line 136] obspy_stages.create_filter_from_stage - INFO: Converting PoleZerosResponseStage electric_dipole_92.000 to a CoefficientFilter
2022-07-29T14:45:11 [line 136] obspy_stages.create_filter_from_stage - INFO: Converting PoleZerosResponseStage electric_si_units to a CoefficientFilter
2022-07-29T14:45:11 [line 136] obspy_stages.create_filter_from_stage - INFO: Converting PoleZerosResponseStage electric_dipole_92.000 to a CoefficientFilter
2022-07-29T14:45:11 [line 136] obspy_stages.create_filter_from_stage - INFO: Converting PoleZerosResponseStage electric_si_units to a CoefficientFilter
2022-07-29T14:45:11 [line 136] obspy_stages.create_filter_from_stage - INFO: Converting PoleZerosResponseStage electric_dipole_92.000 to a CoefficientFilter
2022-07-29T14:45:11 [line 136] obspy_stages.create_filter_from_stage - IN

2022-07-29 14:45:14,540 [line 746] mth5.groups.base.Station.add_run - INFO: run a already exists, returning existing group.
2022-07-29 14:45:14,883 [line 224] mth5.timeseries.run_ts.RunTS.validate_metadata - WARNING: start time of dataset 2020-06-02T19:00:00+00:00 does not match metadata start 2020-06-02T18:41:43+00:00 updating metatdata value to 2020-06-02T19:00:00+00:00
2022-07-29 14:45:22,759 [line 746] mth5.groups.base.Station.add_run - INFO: run b already exists, returning existing group.
2022-07-29 14:45:32,993 [line 746] mth5.groups.base.Station.add_run - INFO: run c already exists, returning existing group.
2022-07-29 14:45:44,959 [line 746] mth5.groups.base.Station.add_run - INFO: run d already exists, returning existing group.
2022-07-29 14:45:46,355 [line 235] mth5.timeseries.run_ts.RunTS.validate_metadata - WARNING: end time of dataset 2020-07-13T19:00:00+00:00 does not match metadata end 2020-07-13T21:46:12+00:00 updating metatdata value to 2020-07-13T19:00:00+00:00


### Examine and Update the MTH5 object

With the open MTH5 Object, we can start to examine what is in it. For example, retrieve the filename and file_version. You can additionally do things such as getting the station information and edit it by setting a new value, in this case the declination model. 

In [11]:
mth5_path = mth5_object.filename

In [12]:
mth5_object

/:
    |- Group: Experiment
    --------------------
        |- Group: Reports
        -----------------
        |- Group: Standards
        -------------------
            --> Dataset: summary
            ......................
        |- Group: Surveys
        -----------------
            |- Group: CONUS_South
            ---------------------
                |- Group: Filters
                -----------------
                    |- Group: coefficient
                    ---------------------
                        |- Group: electric_analog_to_digital
                        ------------------------------------
                        |- Group: electric_dipole_92.000
                        --------------------------------
                        |- Group: electric_si_units
                        ---------------------------
                        |- Group: magnetic_analog_to_digital
                        ------------------------------------
                    |- Group: fap
   

In [13]:
mth5_object.file_version


'0.2.0'

In [14]:
mth5_object.close_mth5()

2022-07-29 14:46:47,622 [line 731] mth5.mth5.MTH5.close_mth5 - INFO: Flushing and closing /home/kkappler/software/irismt/aurora/docs/examples/8P_CAS04.h5


In [15]:
mth5_object = initialize_mth5(mth5_path)

### Need documentation:
Is the cell below required? Or is it an example of some metadata specification?

In [17]:
# Edit and update the MTH5 metadata 
s = mth5_object.get_station("CAS04", survey="CONUS_South")
print(s.metadata.location.declination.model)
s.metadata.location.declination.model = 'IGRF'
print(s.metadata.location.declination.model)
s.write_metadata()    # writes to file mth5_filename

IGRF-13
IGRF


In [18]:
# Print some info about the mth5 
mth5_filename = mth5_object.filename
version = mth5_object.file_version
print(f" Filename: {mth5_filename} \n Version{version}")


 Filename: /home/kkappler/software/irismt/aurora/docs/examples/8P_CAS04.h5 
 Version0.2.0


In [19]:
# Get the available stations and runs from the MTH5 object
mth5_object.channel_summary.summarize()
ch_summary = mth5_object.channel_summary.to_dataframe()
ch_summary

,survey,station,run,latitude,longitude,elevation,component,start,end,n_samples,sample_rate,measurement_type,azimuth,tilt,units,hdf5_reference,run_hdf5_reference,station_hdf5_reference
0,CONUS South,CAS04,a,37.633351,-121.468382,329.3875,ex,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,11267,1.0,electric,13.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
1,CONUS South,CAS04,a,37.633351,-121.468382,329.3875,ey,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,11267,1.0,electric,103.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
2,CONUS South,CAS04,a,37.633351,-121.468382,329.3875,hx,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,11267,1.0,magnetic,13.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
3,CONUS South,CAS04,a,37.633351,-121.468382,329.3875,hy,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,11267,1.0,magnetic,103.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
4,CONUS South,CAS04,a,37.633351,-121.468382,329.3875,hz,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,11267,1.0,magnetic,0.0,90.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
5,CONUS South,CAS04,b,37.633351,-121.468382,329.3875,ex,2020-06-02 22:24:55+00:00,2020-06-12 17:52:23+00:00,847649,1.0,electric,13.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
6,CONUS South,CAS04,b,37.633351,-121.468382,329.3875,ey,2020-06-02 22:24:55+00:00,2020-06-12 17:52:23+00:00,847649,1.0,electric,103.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
7,CONUS South,CAS04,b,37.633351,-121.468382,329.3875,hx,2020-06-02 22:24:55+00:00,2020-06-12 17:52:23+00:00,847649,1.0,magnetic,13.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
8,CONUS South,CAS04,b,37.633351,-121.468382,329.3875,hy,2020-06-02 22:24:55+00:00,2020-06-12 17:52:23+00:00,847649,1.0,magnetic,103.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
9,CONUS South,CAS04,b,37.633351,-121.468382,329.3875,hz,2020-06-02 22:24:55+00:00,2020-06-12 17:52:23+00:00,847649,1.0,magnetic,0.0,90.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>


## If MTH5 file already exists you can start here
If you dont want to execute the previous code to get data again

In [3]:
#interact = False
if interact:
    pass
else:
    h5_path = default_path.joinpath("8P_CAS04.h5")
    mth5_object = mth5.MTH5(file_version="0.1.0")
    #mth5_object.open_mth5(config.stations.local.mth5_path, mode="a")
    mth5_object.open_mth5(h5_path, mode="a")
    ch_summary = mth5_object.channel_summary.to_dataframe()



### Generate an Aurora Configuration file using MTH5 as an input

Up to this point, we have used mth5 and mt_metadata, but haven't yet used aurora. So we will use the MTH5 that we just created (and examined and updated) as input into Aurora.

First, we get a list of the available runs to process from the MTH5 

In [20]:
ch_summary

,survey,station,run,latitude,longitude,elevation,component,start,end,n_samples,sample_rate,measurement_type,azimuth,tilt,units,hdf5_reference,run_hdf5_reference,station_hdf5_reference
0,CONUS South,CAS04,a,37.633351,-121.468382,329.3875,ex,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,11267,1.0,electric,13.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
1,CONUS South,CAS04,a,37.633351,-121.468382,329.3875,ey,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,11267,1.0,electric,103.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
2,CONUS South,CAS04,a,37.633351,-121.468382,329.3875,hx,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,11267,1.0,magnetic,13.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
3,CONUS South,CAS04,a,37.633351,-121.468382,329.3875,hy,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,11267,1.0,magnetic,103.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
4,CONUS South,CAS04,a,37.633351,-121.468382,329.3875,hz,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,11267,1.0,magnetic,0.0,90.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
5,CONUS South,CAS04,b,37.633351,-121.468382,329.3875,ex,2020-06-02 22:24:55+00:00,2020-06-12 17:52:23+00:00,847649,1.0,electric,13.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
6,CONUS South,CAS04,b,37.633351,-121.468382,329.3875,ey,2020-06-02 22:24:55+00:00,2020-06-12 17:52:23+00:00,847649,1.0,electric,103.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
7,CONUS South,CAS04,b,37.633351,-121.468382,329.3875,hx,2020-06-02 22:24:55+00:00,2020-06-12 17:52:23+00:00,847649,1.0,magnetic,13.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
8,CONUS South,CAS04,b,37.633351,-121.468382,329.3875,hy,2020-06-02 22:24:55+00:00,2020-06-12 17:52:23+00:00,847649,1.0,magnetic,103.2,0.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
9,CONUS South,CAS04,b,37.633351,-121.468382,329.3875,hz,2020-06-02 22:24:55+00:00,2020-06-12 17:52:23+00:00,847649,1.0,magnetic,0.0,90.0,digital counts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>


In [21]:
available_runs = ch_summary.run.unique()
sr = ch_summary.sample_rate.unique()
if len(sr) != 1:
    print('Only one sample rate per run is available')
available_stations = ch_summary.station.unique()

In [22]:
available_stations[0]

'CAS04'

### Now, we condense the channel summary into a run_summary.

This method takes an iterable of mth5_objs or h5_paths

## Get a run summary from the mth5

### If the mth5 file is closed, you can get the run summary this way

In [41]:
mth5_run_summary = RunSummary()
h5_path = default_path.joinpath("8P_CAS04.h5")
mth5_run_summary.from_mth5s([h5_path,])
run_summary = mth5_run_summary.clone()
print(run_summary.df)

2022-07-29 14:50:25,346 [line 731] mth5.mth5.MTH5.close_mth5 - INFO: Flushing and closing /home/kkappler/software/irismt/aurora/docs/examples/8P_CAS04.h5


  station_id run_id                     start                       end  \
0      CAS04      a 2020-06-02 19:00:00+00:00 2020-06-02 22:07:46+00:00   
1      CAS04      b 2020-06-02 22:24:55+00:00 2020-06-12 17:52:23+00:00   
2      CAS04      c 2020-06-12 18:32:17+00:00 2020-07-01 17:32:59+00:00   
3      CAS04      d 2020-07-01 19:36:55+00:00 2020-07-13 19:00:00+00:00   

   sample_rate input_channels output_channels  \
0          1.0       [hx, hy]    [ex, ey, hz]   
1          1.0       [hx, hy]    [ex, ey, hz]   
2          1.0       [hx, hy]    [ex, ey, hz]   
3          1.0       [hx, hy]    [ex, ey, hz]   

                               channel_scale_factors  \
0  {'ex': 1.0, 'ey': 1.0, 'hx': 1.0, 'hy': 1.0, '...   
1  {'ex': 1.0, 'ey': 1.0, 'hx': 1.0, 'hy': 1.0, '...   
2  {'ex': 1.0, 'ey': 1.0, 'hx': 1.0, 'hy': 1.0, '...   
3  {'ex': 1.0, 'ey': 1.0, 'hx': 1.0, 'hy': 1.0, '...   

                                           mth5_path  
0  /home/kkappler/software/irismt/aurora/d

In [43]:
run_summary.df

,station_id,run_id,start,end,sample_rate,input_channels,output_channels,channel_scale_factors,mth5_path
0,CAS04,a,2020-06-02 19:00:00+00:00,2020-06-02 22:07:46+00:00,1.0,"[hx, hy]","[ex, ey, hz]","{'ex': 1.0, 'ey': 1.0, 'hx': 1.0, 'hy': 1.0, '...",/home/kkappler/software/irismt/aurora/docs/exa...
1,CAS04,b,2020-06-02 22:24:55+00:00,2020-06-12 17:52:23+00:00,1.0,"[hx, hy]","[ex, ey, hz]","{'ex': 1.0, 'ey': 1.0, 'hx': 1.0, 'hy': 1.0, '...",/home/kkappler/software/irismt/aurora/docs/exa...
2,CAS04,c,2020-06-12 18:32:17+00:00,2020-07-01 17:32:59+00:00,1.0,"[hx, hy]","[ex, ey, hz]","{'ex': 1.0, 'ey': 1.0, 'hx': 1.0, 'hy': 1.0, '...",/home/kkappler/software/irismt/aurora/docs/exa...
3,CAS04,d,2020-07-01 19:36:55+00:00,2020-07-13 19:00:00+00:00,1.0,"[hx, hy]","[ex, ey, hz]","{'ex': 1.0, 'ey': 1.0, 'hx': 1.0, 'hy': 1.0, '...",/home/kkappler/software/irismt/aurora/docs/exa...


## Otherwise, work from the mth5_object

In [ ]:
mth5_run_summary = RunSummary()
mth5_run_summary.from_mth5s([mth5_object,])
run_summary = mth5_run_summary.clone() # intended to act as an "aurora" run summary, 
#which can be manipulated ... 
print(run_summary.df)

### Here are the columns in the run summary

In [ ]:
print(run_summary.df.columns)

### Here are the columns to look at to get a sense of the data coverage

In [ ]:
coverage_short_list_columns = ['station_id', 'run_id', 'start', 'end', ]
print(run_summary.df[coverage_short_list_columns])

### Based on the available data, make a KernelDataset

Syntax is kernel_dataset.from_run_summary(run_summary, local_station_id, reference_station_id)

In [ ]:
kernel_dataset = KernelDataset()
kernel_dataset.from_run_summary(run_summary, "CAS04")
print(kernel_dataset.df[coverage_short_list_columns])

### By default, all runs will be processed


To restrict to processing a single run, or a list of runs, we can either tell KernelDataset to keep or drop a station_run dictionary. 


## Here is one way to select a single run:


In [32]:
station_runs_dict = {}
station_runs_dict["CAS04"] = ["a", ]
keep_or_drop = "keep"

kernel_dataset.select_station_runs(station_runs_dict, keep_or_drop)
print(kernel_dataset.df[coverage_short_list_columns])

  station_id run_id                     start                       end
0      CAS04      a 2020-06-02 19:00:00+00:00 2020-06-02 22:07:46+00:00


## To discard runs that are not very long

In [33]:
kernel_dataset = KernelDataset()
kernel_dataset.from_run_summary(run_summary, "CAS04")
cutoff_duration_in_seconds = 15000
kernel_dataset.drop_runs_shorter_than(cutoff_duration_in_seconds)
print(kernel_dataset.df[coverage_short_list_columns])

  station_id run_id                     start                       end
0      CAS04      b 2020-06-02 22:24:55+00:00 2020-06-12 17:52:23+00:00
1      CAS04      c 2020-06-12 18:32:17+00:00 2020-07-01 17:32:59+00:00
2      CAS04      d 2020-07-01 19:36:55+00:00 2020-07-13 19:00:00+00:00


### Select only runs "b" & "d"

In [34]:
kernel_dataset = KernelDataset()
kernel_dataset.from_run_summary(run_summary, "CAS04")
station_runs_dict = {}
station_runs_dict["CAS04"] = ["b","d"]
keep_or_drop = "keep"
kernel_dataset.select_station_runs(station_runs_dict, keep_or_drop)
print(kernel_dataset.df[coverage_short_list_columns])

  station_id run_id                     start                       end
0      CAS04      b 2020-06-02 22:24:55+00:00 2020-06-12 17:52:23+00:00
1      CAS04      d 2020-07-01 19:36:55+00:00 2020-07-13 19:00:00+00:00


### The same result can be obtained by _excluding_ runs a & c

In [35]:
kernel_dataset = KernelDataset()
kernel_dataset.from_run_summary(run_summary, "CAS04")
station_runs_dict = {}
station_runs_dict["CAS04"] = ["a","c"]
keep_or_drop = "drop"
kernel_dataset.select_station_runs(station_runs_dict, keep_or_drop)
print(kernel_dataset.df[coverage_short_list_columns])

  station_id run_id                     start                       end
0      CAS04      b 2020-06-02 22:24:55+00:00 2020-06-12 17:52:23+00:00
1      CAS04      d 2020-07-01 19:36:55+00:00 2020-07-13 19:00:00+00:00


Make an aurora configuration file (and then save that json file.)

In [36]:
cc = ConfigCreator()#config_path=CONFIG_PATH)
config = cc.create_run_processing_object(emtf_band_file=BANDS_DEFAULT_FILE,
                                    sample_rate=sr[0]
                                    )
config.stations.from_dataset_dataframe(kernel_dataset.df)
for decimation in config.decimations:
    decimation.estimator.engine = "RME"

Take a look at the config:

In [37]:
config

{
    "processing": {
        "decimations": [
            {
                "decimation_level": {
                    "anti_alias_filter": "default",
                    "bands": [
                        {
                            "band": {
                                "decimation_level": 0,
                                "frequency_max": 0,
                                "frequency_min": 0,
                                "index_max": 30,
                                "index_min": 25
                            }
                        },
                        {
                            "band": {
                                "decimation_level": 0,
                                "frequency_max": 0,
                                "frequency_min": 0,
                                "index_max": 24,
                                "index_min": 20
                            }
                        },
                        {
                            "band": {


Run the Aurora Pipeline using the input MTh5 and Confiugration File

In [39]:
show_plot = True
tf_cls = process_mth5(config,
                    kernel_dataset,
                    units="MT",
                    show_plot=show_plot,
                    z_file_path=None,
                    return_collection=False
                )

2022-07-29 14:49:34,821 [line 1192] mth5.mth5.MTH5.get_run - ERROR: Need to input 'survey' for file version 0.2.0


Processing config indicates 4 decimation levels 


ValueError: Need to input 'survey' for file version 0.2.0

In [28]:

type(tf_cls)

mt_metadata.transfer_functions.core.TF

Write the transfer functions generated by the Aurora pipeline

In [29]:
 tf_cls.write_tf_file(fn="emtfxml_test.xml", file_type="emtfxml")

2022-06-25 08:19:55,862 [line 197] mt_metadata.transfer_functions.io.readwrite.write_file - INFO: Wrote emtfxml_test.xml


EMTFXML(station='CAS04', latitude=37.63, longitude=-121.47, elevation=329.39)

In [28]:
tf_cls.write_tf_file(fn="emtfxml_test.xml", file_type="edi")

2022-06-03 17:58:39,327 [line 197] mt_metadata.transfer_functions.io.readwrite.write_file - INFO: Wrote emtfxml_test.xml


EMTFXML(station='CAS04', latitude=37.63, longitude=-121.47, elevation=329.39)

In [29]:
 tf_cls.write_tf_file(fn="emtfxml_test.xml", file_type="zmm")

2022-06-03 17:58:39,876 [line 197] mt_metadata.transfer_functions.io.readwrite.write_file - INFO: Wrote emtfxml_test.xml


EMTFXML(station='CAS04', latitude=37.63, longitude=-121.47, elevation=329.39)